# SIT320 Advanced Algorithms
## Task 11



##Activity 1
###Karger’s algorithm finds a minimum cut based on the number of edges. What if we have weights on the edges. How would you modify Karger’s algorithm (or design a new algorithm) to take into account edge weights?
- You are not expected to write any code here but design an algorithm. You can present your algorithm in form of a pseudo-code, paragraph, or video recording.
- You are expected to make any reasonable assumptions about the problem, to simplify your formulation of proposed algorithm.

##### References used include CS:5350 Karger’s Mincut Algorithm by Asad Mahmood [1] and SIT320 Advanced Algorithms Deakin [2].

#### Karger’s Algorithm
Karger’s algorithm finds global minimum cuts, it separates a graph into two parts while minimising the total weight of all edges that need to be cut (except for s and t that need to be separated).
- Minimum cut = smallest number of edges to be removed for graph disconnection.
- This works for unweighted graphs and for graph with positive edge weights (edges selected for contraction with probability proportional to their weight).
- The randomness can produce a different result on the same graph, with high probability.
- By running it n^2 times, likely to find at least one minimum cut.

1. Select an edge at random
2. Then contact the chosen edge
3. Repeat until only two nodes are left


> Function Karger(G) <br>
>>// While there are more than 2 supernodes <br>
for $i$ &#8592; $1$ to $n - 2$ do <br>
Pick an edge $(u, v)$ in $G$ uniformly at random <br>
// Merge $u$ and $v$ <br>
$G$ &#8592; CONTRACT $(G, {u, v})$ <br>
return Edges between two vertices that remain <br>
end function <br>


### Modified Karger’s Algorithm (Weighted Edges)
#### Aim: Modify Karger’s algorithm to hand weighted edges. Need to adjust how edges are selected and contracted (merging two adjacent $v$ (nodes) in graph into single vertex, while maintaining connectivity of graph).

##### Assumptions:
- Need to adjust the probability distribution for selecting edges.
- Edges are chosen uniformly at random, but weighted edges we want to select edges with higher weights more often.
- We have an undirected, connected graph with weighted edges.
- All edge weights are positive integers, chosen randomly - represent the cost of removing that edge.
- No self-loops are present in the graph.

#### Pseudo-code
Input: Connected, undirected graph $G$ with weighted edges, $G = (V, E) <br>
Output: Two nodes representing min-cut (weighted) and the weight of cut <br>

> Function Karger_weights(G)
>// Initialise variables for edge selection <br> cumulative_weight <br>
selected_edge <br>
random_weight <br><br>
// While there are more than 2 supernodes
1. while number of supernodes in $G > 2$ do:
2. total_weight = sum of weights of all edges in $G$<br>
3. random_weight = random num between 0 and total_weight<br> <br>

>// Iterate through all edges to select based on weights
for each edge $(u, v) in G$ do:
1. Collect edge weight to the cumulative weight
2. Cumulative_weight += weight of $(u, v)$ <br>
if cumulative_weight >= random_weight<br>
>>Selected_edge = $(u, v)$ <br>
// Merge $u, v$ <br>
$G$ &#8592; CONTRACT $(G, {u, v}) $<br>
>return Edges between two vertices that remain
end function


### Time Complexity
Karger's Algorithm (without weights), merge $O(n^2)$, for $n$ nodes (single run) $O(n^2)$, optimal solution needs $n^2 log(n)$ runs, thus time complexity = $O(n^4 log(n))$. <br>

**Modified Karger's Algorithm ** (with weights), selecting an edge is $O(E)$, $E$ the number of edges. Each step is repeated $n-2$ contractions (merging) where $n$ is the number of nodes, overall complexity is $O((n-2)*E)$, resulting in $O(n*log(E))$.
### Analysis
This task enabled me to understand Karger’s Algorithm and how the min-cut (random selection) functions. What was interesting was the effects of running the algorithm through different iterations. The modified algorithm ensures that edges with higher weights have a greater chance of being selected for merging (contraction), in finding min-cut.
1. Find total weight of all edges in $G$, select an edge randomly based on weights of edges.
- Ensures edges with higher weights are more likely to selected.
2. Keep track of the cumulative weights of edges and use random value to select an edge.
3. Min-cut value is calculated by summing the weight of the remaining edges after contraction (merging). <br>

Note: The Karger-Stein algorithm may be better suited as it reduces time complexity to $O(n^2)$, by reducing the nodes to $n/sqrt(2)$ [1].


##Activity 2

###Write code for Ford-fulkerson algorithm and apply it on the following problem
- Your algorithm should print the values of max flow and min cut.
- Make sure your algorithm prints the edges involved in the minimum-cut.
- You are welcome to write code from scratch, but are allowed to look over the internet, e.g., hSps://www.programiz.com/dsa/ford-fulkerson-algorithm. If you are using code from the internet or chatGPT, make sure you document it that demonstrates your understanding of the code. Your tutor will discuss the salient features of the code with you.


### The Ford-fulkerson Algorithm
#### Finds min-cuts and max flows.
- Graphs are directed and edges have capacities (weights).
- Source (vertex $s$) and sink (vertex $t$)
- $s$ have only outgoing edges
- $t$ has only incoming edges<br>

A min s-t cut = a cut that separates $s$ from $t$ with minimum capacity, <br>
Each edge has a flow =
1. The amount of stuff coming out of $s$
2. The amount of stuff flowing into $t$

- Flow on an edge must be less than its capacity
- At each vertex the incoming flows must equal the outgoing flows
- unmarked edges = 0 <br>

A maximum flow = is a flow of maximum value
- The value of max flow from $s$ to $t$ = the cost of a min s-t cut. <br>

I modified the code given in the lecture as it didn’t print the min cut or edges and most importantly needed to find a way of augmenting paths in the residual graph $G_f$. I used the pseudocode to create a class for the Ford-fulkerson algorithm to create more structure in the code. <br>

### Original Ford-fulkerson Algorithm Pseudocode
Input: connected, undirected graph $G$ with weighted edges, $G = (V, E)$ <br>
Output: flow $f=$ optimal/ maximal flow values form $s$ to $t$ in $G_f$ <br>

>Ford-fulkerson(G) <br>
// Start with zero flow <br>
$f $ & &#8592; all zeros
$G_f$ &#8592; $G$ <br>
while $t$ is reachable from $s$ to $t$ in $G_f$ <br>

>>// Repeat process of finding paths between $s$ and $t$ in residual graph until there is none. <br>
$f$ &#8592; increase_flow$(p, f)$ <br>
update $G_f$ <br>
return $f$



In [17]:
# Ford-fulkerson Algorithm
# Augmenting path = path from the source node (s) to the sink node in residual graph
# Residual graph = G_f

# Modules
from collections import defaultdict, deque
import time
import random

# Class
'''
Inputs: Graph (network as an adjacency list)
        Each node if represented as target, capacity, flow
        Source: The root node of the network
        Sink: Target node of network
'''
class FordFulkerson:
  def __init__(self, graph, source, sink, max_iterations):
      self.graph = graph # Original graph (network)
      self.source = source # source set (s)
      self.sink = sink # sink set (t)
      self.max_flow = 0
      self.visited = set()
      # Added for test case
      self.max_iterations = max_iterations

# Method find augmenting path in the residual graph (G_f)
# Using Breadth First Search O(nm^2)
# Push flow to increase the total flow in the network
  def find_path(self):
    '''
    parent: dictionary to track parent node of each visisted node during BFS
    queue: A dequeue (double ended queue) for BFS traversal
    self.visisted: Set to keep track of visisted nodes, cleared at start of BFS
    '''
    parent = {}
    queue = deque()
    queue.append(self.source)
    self.visited.clear()

    # BFS traversal to search for augmenting path
    while queue:
      u = queue.popleft()
      self.visited.add(u)

      # Each node u popped from queue, Neighbors v in g_f
      # Iterate over edges connected to node u
      for v, capacity, flow in self.graph[u]:
        # node u not visited available capacity on edge
        if v not in self.visited and capacity - flow > 0:
          # u = parent of v to reconstruct path later
          parent[v] = (u, capacity, flow) # Updates parent

          # If both conditions met for v, start BFS
          # IF ONLY v = sink, follow parent pointers backward to source & record edge & min capacity
          if v == self.sink: # Found a path from s to t
            # Return path (edges), current neighbor & and min capacity
            path = []
            current = v
            min_capacity = float('inf')
            # Reconstruct path from sink node to source node
            # Calculate min capacity for that path
            while current != self.source:
              u, capacity, flow = parent[current] # Edge info from parent
              # Compare current min capcity with reminaing capacity and update
              min_capacity = min(min_capacity, capacity - flow) # Calculate min capacity along current edge
              path.append((u, current))
              current = u
            print("Found augmenting path:", path)  # Path found
            return path, min_capacity

          queue.append(v) # Else v add to queue to re expore (BFS)

    print("No augmenting path found")  # Check Paths
    return None, 0 # No augmenting path found

# Method find augmenting paths and update flow until no augmenting paths remain
# Each iteration calls find_path() which uses BFS O(V+E), vertices and num edges
  def ford_fulkerson(self):
    # Loops until find_path method (BFS) can find an augmenting path
    # DFS determins whether there are any paths left to explore
    elapsed_time = 0
    while True:
      self.visited.clear()  # Clear visited nodes
      augmenting_path, min_capacity = self.find_path()
      start_time = time.time() # Timer start

      if not augmenting_path:
        break

      print(f"Updating flow on path: {augmenting_path}, Flow amount: {min_capacity}")

      for u, v in augmenting_path:
        # Update edges (forward and backward)
        for edge in self.graph[u]:
          if edge[0] == v:
            edge[2] += min_capacity
            break
        for edge in self.graph[v]:
          if edge[0] == u:
            edge[2] -= min_capacity
            break

      self.max_flow += min_capacity
      end_time = time.time() # Timer End
      elapsed_time = end_time - start_time
    print(f"Max Flow = {self.max_flow}, Elapsed Time = {elapsed_time} seconds")
    self.min_cut_edges = [] # Clear edges for next case

# Method finds the min-cut of the graph flow (network) from s to t
  def min_cut(self):

    # visited set keeps track of nodes visited by DFS traversal
    self.visited.clear()
    # DFS used to find nodes in s
    # Starts at s and finds all reachable nodes in G_f
    self.dfs(self.source)

    min_cut_edges = [] # Stores all edges
    for u in self.visited: # Check all nodes visited in DFS s of min-cut
      # Loop to compare edges connected to node (u) in graph
      for v, capacity, flow in self.graph[u]:
        # If not visited, v = part of t reachable from u but not s
        if v not in self.visited: # in t, add to min_cut_list list
          min_cut_edges.append((u, v))

    return min_cut_edges # All edges belong to min-cut

# Method runs Depth First Search to find nodes/ valid augmenting path s in G_f
# Nodes that are neighbors from source node in residual graph
# Identifies the nodes in s of min-cut
# Terminates the Ford-Fulkerson algorithm when no augementing path from source to sink can be found in residual graph
  def dfs(self, u):
    self.visited.add(u)
    for v, capacity, flow in self.graph[u]:
      if v not in self.visited and capacity - flow > 0:
        self.dfs(v)



# Test Case 1
### Test a simple graph

In [18]:
# Test Case
# Define graph (adjacency list)
# Each edge = target, capacity, flow
graph = {
    's': [['a', 10, 0], ['b', 1, 0]],
    'a': [['b', 15, 0], ['t', 3, 0]],
    'b': [['t', 10, 0]],
    't': []
}

source = 's'
sink = 't'
max_iterations = 1
ff = FordFulkerson(graph, source, sink, max_iterations)
ff.ford_fulkerson()  # Ford-Fulkerson

# Get the maximum flow
max_flow = ff.max_flow
# Find the minimum cut edges
min_cut = ff.min_cut()

print("Min Cut Edges:", min_cut)

Found augmenting path: [('a', 't'), ('s', 'a')]
Updating flow on path: [('a', 't'), ('s', 'a')], Flow amount: 3
Found augmenting path: [('b', 't'), ('a', 'b'), ('s', 'a')]
Updating flow on path: [('b', 't'), ('a', 'b'), ('s', 'a')], Flow amount: 7
Found augmenting path: [('b', 't'), ('s', 'b')]
Updating flow on path: [('b', 't'), ('s', 'b')], Flow amount: 1
No augmenting path found
Max Flow = 11, Elapsed Time = 0.00044274330139160156 seconds
Min Cut Edges: [('s', 'a'), ('s', 'b')]


## Test Case 2
### Test flow from s to t (different graph)

In [19]:
graph1 = {
    's': [['a', 10, 0], ['b', 10, 0], ['c', 10, 0]],
    'a': [['d', 5, 0]],
    'b': [['d', 10, 0]],
    'c': [['t', 10, 0]],
    'd': [['t', 15, 0]],
    't': []
}

ff1 = FordFulkerson(graph1, source, sink, max_iterations)
ff1.ford_fulkerson()
max_flow1 = ff1.max_flow
min_cut1 = ff1.min_cut()
print("Min Cut Edges:", min_cut1)


Found augmenting path: [('c', 't'), ('s', 'c')]
Updating flow on path: [('c', 't'), ('s', 'c')], Flow amount: 10
Found augmenting path: [('d', 't'), ('b', 'd'), ('s', 'b')]
Updating flow on path: [('d', 't'), ('b', 'd'), ('s', 'b')], Flow amount: 10
Found augmenting path: [('d', 't'), ('a', 'd'), ('s', 'a')]
Updating flow on path: [('d', 't'), ('a', 'd'), ('s', 'a')], Flow amount: 5
No augmenting path found
Max Flow = 25, Elapsed Time = 2.2172927856445312e-05 seconds
Min Cut Edges: [('a', 'd'), ('s', 'b'), ('s', 'c')]


## Test Case 3
### A disconnected graph (no flow possible)

In [20]:
# A disconnected graph (no flow possible)
graph2 = {
    's': [('a', 10, 0)],
    'a': [],
    't': [('b', 5, 0)],
    'b': []
}

ff2 = FordFulkerson(graph2, source, sink, max_iterations)
ff2.ford_fulkerson()
max_flow2 = ff2.max_flow
min_cut2 = ff2.min_cut()
print("Max Flow:", max_flow2)
print("Min Cut Edges:", min_cut2)


No augmenting path found
Max Flow = 0, Elapsed Time = 0 seconds
Max Flow: 0
Min Cut Edges: []


## Test Case 4
### Complex Graph and Change in Capacity/ flow test

In [21]:
# Define a complex graph for testing
complex_graph = {
    's': [['a', 4, 0], ['b', 2, 0], ['c', 6, 0]],
    'a': [['e', 6, 0], ['d', 3, 0]],
    'b': [['e', 3, 0], ['f', 3, 0]],
    'c': [['f', 10, 0]],
    'd': [['t', 4, 0]],
    'e': [['t', 4, 0]],
    'f': [['t', 4, 0]],
    't': []
}

# Number of iterations and reset interval
max_iterations = 100
reset_interval = 10

ff_complex = FordFulkerson(complex_graph, source, sink, max_iterations)
ff_complex.ford_fulkerson()
max_flow3 = ff_complex.max_flow
min_cut3 = ff_complex.min_cut()
print("Max Flow:", max_flow3)
print("Min Cut Edges:", min_cut3)

print("")
print("Change capacity and flow b -> e")
# Increase capacity on edge from 'b' to 'e'
complex_graph['b'][0] = ['e', 5, 0]

# Decrease initial flow on edge from 's' to 'a'
complex_graph['s'][0] = ['a', 3, 0]  # Decreased flow from 4 to 3
ff_complex.ford_fulkerson()
max_flow3 = ff_complex.max_flow
min_cut3 = ff_complex.min_cut()
print("Max Flow:", max_flow3)
print("Min Cut Edges:", min_cut3)

Found augmenting path: [('e', 't'), ('b', 'e'), ('s', 'b')]
Updating flow on path: [('e', 't'), ('b', 'e'), ('s', 'b')], Flow amount: 2
Found augmenting path: [('e', 't'), ('a', 'e'), ('s', 'a')]
Updating flow on path: [('e', 't'), ('a', 'e'), ('s', 'a')], Flow amount: 2
Found augmenting path: [('d', 't'), ('a', 'd'), ('s', 'a')]
Updating flow on path: [('d', 't'), ('a', 'd'), ('s', 'a')], Flow amount: 2
Found augmenting path: [('f', 't'), ('c', 'f'), ('s', 'c')]
Updating flow on path: [('f', 't'), ('c', 'f'), ('s', 'c')], Flow amount: 4
No augmenting path found
Max Flow = 10, Elapsed Time = 2.0503997802734375e-05 seconds
Max Flow: 10
Min Cut Edges: [('f', 't'), ('s', 'a'), ('s', 'b')]

Change capacity and flow b -> e
Found augmenting path: [('d', 't'), ('a', 'd'), ('s', 'a')]
Updating flow on path: [('d', 't'), ('a', 'd'), ('s', 'a')], Flow amount: 1
No augmenting path found
Max Flow = 11, Elapsed Time = 2.3603439331054688e-05 seconds
Max Flow: 11
Min Cut Edges: [('a', 'd'), ('s', 'b'

### Ford-fulkerson Algorithm Analyses
If there is no max capacity, capacity is  C, then algorithm can take >= C iterations
- Some choices lead to exponential algorithms, but smart choices lead to polynomial algorithms.
- If edges capacities are irrational, ‘no guarantee that Ford-Fulkerson terminates (or converges to a maximum flow)’ [8]. I used BFS to find the augmenting path with fewest edges [8].

### Resolving Issues
If I had more time for this task implementing an adaptive strategy, where the algorithm adjusts behaviour based on past performance may reduce the number of iterations and still find the most optimal path. I create a random augmentation path selection and kept track of which iteration was the best for max flow. One issue with this code is that it assumes the initial flow on all edges is 0, for future applications this may need to be modified to be more flexible. I also had an issue with clearing visited nodes after each different runs of the algorithm which I fixed.


### Ford-Fulkerson Time Complexity
Normal running time is O(mnC) <br>
I use BFS to find the augmenting path with fewest edges, based on Edmonds-Karp [8] and DFS to create time complexity of O(V*E^2).


#### References
- [1]	CLRS, et al, “Introduction to Algorithms,” in Chapter 26 Maximum Flow, Cambridge, Massachusetts, MIT Press Academic, 2022.
- [2]	H. Satopay, “The Ultimate Markdown Guide (for Jupyter Notebook),” 18 11 2019. [Online]. Available: https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd. [Accessed 23 9 2023].
- [3]	A. Mahood, “CS:5350 Karger’s Mincut Algorithm,” The University of Iowa, Iowa, USA, 2019.
- [4]	Deakin University, “Module 11: Network-based Algorithms,” Deakin University, Melbourne, Australia, 2023.
- [5]	E. Maheshwari, “Karger’s Algorithm,” Medium, 24 6 2018. [Online]. Available: https://medium.com/@dev.elect.iitd/kargers-algorithm-d8067eb1b790. [Accessed 20 9 2023].
